# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

u'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


[(None, None, None, None, None, 3, u'Six', u'Paul', u'Alsace'),
 (None, None, None, None, None, 6, u'Marmagne', u'Bernard', u'Bourgogne'),
 (None, None, None, None, None, 8, u"Lioger d'Harduy", u'Gabriel', u'Bourgogne'),
 (None, None, None, None, None, 16, u'Barbin', u'Bernard', u'Bourgogne'),
 (None, None, None, None, None, 17, u'Faiveley', u'Guy', u'Bourgogne'),
 (None, None, None, None, None, 18, u'Tramier', u'Jean', u'Bourgogne'),
 (None, None, None, None, None, 19, u'Dupaquier', u'Roger', u'Bourgogne'),
 (None, None, None, None, None, 20, u'Lamy', u'Jean', u'Bourgogne'),
 (None, None, None, None, None, 21, u'Cornu', u'Edmond', u'Bourgogne'),
 (None, None, None, None, None, 26, u'Violot', u'Gilbert', u'Bourgogne')]

* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master2
WHERE NB=5;

 * sqlite:///wine.db
Done.


[(9, u'Chapelle Chambertin', 11.9, 1973, u'1982-03-26', u'LYON', 20, 5, u'Audiberti', u'Jacques', u'petit', u'  AUVERGNE-RH\xd4NE-ALPES'),
 (10, u'Beaujolais Villages', 11.8, 1979, u'1983-06-06', u'LYON', 10, 5, u'Audiberti', u'Jacques', u'petit', u'  AUVERGNE-RH\xd4NE-ALPES'),
 (56, u'Saint Amour', None, 1974, u'1983-05-05', u'ROCQUENCOURT', 80, 5, u'Audiberti', u'Jacques', u'petit', u' \xceLE-DE-FRANCE'),
 (100, u'Seyssel', 11, 1974, u'1983-02-21', u'CAEN', 3, 5, u'Audiberti', u'Jacques', u'petit', u' NORMANDIE')]

---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [53]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
VINES:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [10]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

Done.
Done.
Done.


[(11.5,),
 (11.3,),
 (12.1,),
 (10.9,),
 (11.7,),
 (11.2,),
 (12.3,),
 (11.9,),
 (11.8,),
 (10.7,),
 (11.4,),
 (10.5,),
 (11,),
 (11.6,),
 (12,),
 (10,),
 (12.5,),
 (10.8,),
 (14,),
 (11.1,),
 (13,)]

Notice that we use `NULL` given that `None` is only used for display.

In [11]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [7]:
%%sql

DROP TABLE IF EXISTS WINES;
DROP TABLE IF EXISTS WINES1;
DROP TABLE IF EXISTS PRODUCERS;
DROP TABLE IF EXISTS PRODUCTIONS;

CREATE TABLE WINES1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL
UNION
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

CREATE TABLE WINES AS
SELECT DISTINCT *
FROM WINES1;
DROP TABLE IF EXISTS WINES1;

CREATE TABLE PRODUCERS AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

CREATE TABLE PRODUCTIONS AS
SELECT DISTINCT NP, NV, QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;



 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [8]:
%%sql

DROP TABLE IF EXISTS CUSTOMERS;
DROP TABLE IF EXISTS REGIONS;
DROP TABLE IF EXISTS SALES;

CREATE TABLE CUSTOMERS AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

CREATE TABLE REGIONS AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2
WHERE LIEU IS NOT NULL;

CREATE TABLE SALES AS
SELECT DISTINCT DATES, LIEU, NB, NV, QTE
FROM MASTER2
WHERE DATES IS NOT NULL AND LIEU IS NOT NULL AND NB IS NOT NULL AND NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [9]:
%%sql
SELECT DISTINCT TYPE
FROM CUSTOMERS;

 * sqlite:///wine.db
Done.


[(u'petit',), (u'moyen',), (u'gros',)]

#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [10]:
%%sql

SELECT DISTINCT CRU,REGION
FROM (SELECT CRU, NP AS N
      FROM WINES 
      JOIN PRODUCTIONS 
      ON WINES.NV=PRODUCTIONS.NV)
JOIN PRODUCERS 
ON NP = N
WHERE CRU = "Pommard" OR CRU = "Brouilly";

 * sqlite:///wine.db
Done.


[(u'Pommard', u'Bourgogne'),
 (u'Pommard', u'Rhone'),
 (u'Brouilly', u'Bourgogne')]

#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [11]:
%%sql

SELECT DISTINCT REGION
FROM (SELECT CRU, NP AS N
      FROM WINES 
      JOIN PRODUCTIONS 
      ON WINES.NV=PRODUCTIONS.NV)
JOIN PRODUCERS 
ON NP = N
WHERE CRU = "Pommard" 

INTERSECT

SELECT DISTINCT REGION
FROM (SELECT CRU, NP AS N
      FROM WINES 
      JOIN PRODUCTIONS 
      ON WINES.NV=PRODUCTIONS.NV)
JOIN PRODUCERS 
ON NP = N
WHERE CRU = "Brouilly" 

 * sqlite:///wine.db
Done.


[(u'Bourgogne',)]

#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [12]:
%%sql

SELECT DISTINCT CRU, MILL, SUM(QTE)
FROM SALES
JOIN WINES
ON WINES.NV = SALES.NV
GROUP BY CRU, MILL

 * sqlite:///wine.db
Done.


[(u'Arbois', 1980, 8),
 (u'Auxey Duresses', 1914, 80),
 (u'Beaujolais Primeur', 1983, 7),
 (u'Beaujolais Villages', 1975, 10),
 (u'Beaujolais Villages', 1976, 120),
 (u'Beaujolais Villages', 1978, 130),
 (u'Beaujolais Villages', 1979, 520),
 (u'Chapelle Chambertin', 1973, 30),
 (u'Chateau Corton Grancey', 1980, 4),
 (u'Chenas', 1984, 1),
 (u'Clairette de Die', 1976, 5),
 (u'Cornas', 1980, 25),
 (u'Cote de Brouilly', 1980, 23),
 (u'Coteaux du Tricastin', 1974, 1),
 (u'Coteaux du Tricastin', 1978, 12),
 (u'Cotes de Beaune Villages', 1971, 14),
 (u'Cotes de Beaune Villages', 1975, 96),
 (u'Cotes de Nuits', 1978, 21),
 (u'Cotes de Provence', 1980, 18),
 (u'Etoile', 1980, 9),
 (u'Fleurie', 1980, 7),
 (u'Gentil', 1975, 36),
 (u'Gigondas', 1978, 5),
 (u'Grands Echezeaux', 1968, 31),
 (u'Julienas', 1974, 2),
 (u'Mercurey', 1980, 57),
 (u'Mercurey', 1981, 36),
 (u'Meursault', 1980, 2),
 (u'Monthelie', 1968, 106),
 (u'Pommard', 1972, 23),
 (u'Pommard', 1976, 55),
 (u'Pouilly Vinzelles', 1968, 10),
 (u'Rasteau', 1980, 15),
 (u'Richebourg', 1943, 72),
 (u'Saint Amour', 1974, 80),
 (u'Saint Amour', 1980, 14),
 (u'Savigny les Beaunes', 1974, 44),
 (u'Savigny les Beaunes', 1977, 7),
 (u'Savigny les Beaunes', 1978, 11),
 (u'Seyssel', 1974, 24),
 (u'Seyssel', 1980, 21),
 (u'Tavel', 1980, 33),
 (u'Volnay', 1980, 5)]

#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [13]:
%%sql

SELECT NV, Number_of_producers
FROM (SELECT NV, COUNT(NP) AS Number_of_producers
    FROM PRODUCTIONS
    GROUP BY NV)
WHERE Number_of_producers >3;

 * sqlite:///wine.db
Done.


[(45, 5), (78, 5), (89, 4), (98, 5)]

#### Exercise 2.6

Which producers have not produced any wine?

In [14]:
%%sql

SELECT NP, NOM, PRENOM
FROM PRODUCERS P
WHERE NOT EXISTS(SELECT *
                FROM PRODUCTIONS
                WHERE P.NP = PRODUCTIONS.NP)

 * sqlite:///wine.db
Done.


[(3, u'Six', u'Paul'),
 (6, u'Marmagne', u'Bernard'),
 (8, u"Lioger d'Harduy", u'Gabriel'),
 (16, u'Barbin', u'Bernard'),
 (17, u'Faiveley', u'Guy'),
 (18, u'Tramier', u'Jean'),
 (19, u'Dupaquier', u'Roger'),
 (20, u'Lamy', u'Jean'),
 (21, u'Cornu', u'Edmond'),
 (26, u'Violot', u'Gilbert'),
 (28, u'Monnier', u'Rene'),
 (29, u'de Suremain', u'Robert'),
 (31, u'Prudhon', u'Henri'),
 (32, u'Mestre', u'Philippe'),
 (33, u'Prieur', u'Guy'),
 (34, u'Monnier', u'Rene'),
 (37, u'Delagrange Bachelet', None),
 (39, u'de Suremin', u'Hugues'),
 (41, u'Juillot', u'Michel'),
 (44, u'Feyveley', None),
 (47, u'Thenard', None),
 (48, u'Flamand', None),
 (49, u'Guillot', u'Pierre'),
 (51, u'Simonet', u'Leon'),
 (53, u'Cooperative', None),
 (57, u'Desmules', u'Pierre'),
 (58, u'Joubert', u'Claude'),
 (61, u'Champier', u'Charles'),
 (62, u'Champagnon', u'Louis'),
 (66, u'Cooperative', None),
 (77, u'Tissot', u'Andre'),
 (83, u'Goy', None),
 (91, u'Chave', u'Gerard'),
 (92, u'De Boissieu', None),
 (94, u'Grippat', u'Jean Louis'),
 (96, u'Boisson', u'Jean Pierre'),
 (97, u'Quiot', u'Pierre'),
 (100, u'Bernard', u'Georges'),
 (101, u'Verdan', u'Antoine'),
 (102, u'Cooperative', None),
 (103, u'Coulouvrat', u'Michel'),
 (104, u'Rabasse Chavarin', u'Abel'),
 (105, u'Roux', u'Charles'),
 (106, u'Alary', None),
 (108, u'Archimbaud', u'Jean'),
 (109, u'Rey', u'Marcel'),
 (110, u'Cooperative', None),
 (111, u'Zafiropulo', u'Costa'),
 (112, u'Prebost', u'Gaston'),
 (113, u'Gomez', u'Rene'),
 (114, u'Brunet', u'Georges'),
 (115, u'Bordonado', u'Raymond'),
 (116, u'Imbert', u'Christian'),
 (117, u'Cooperative', None),
 (118, u'Cooperative', None),
 (119, u'Cooperative', None),
 (121, u'Cooperative', None),
 (122, u'Dupuy', u'Charles'),
 (123, u'Laurent', u'Pierre'),
 (124, u'Daurat Fort', u'Robert'),
 (125, u'Baldes', u'Jean'),
 (126, u'Jouffreau', u'Jean'),
 (127, u'Brumont', u'Alban'),
 (129, u'Barrere', u'Alfred')]

#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [15]:
%%sql

SELECT NB, NOM, PRENOM
FROM CUSTOMERS AS C
WHERE 0<(SELECT COUNT(WINES.NV)
        FROM WINES
        JOIN SALES 
        ON WINES.NV = SALES.NV
        WHERE SALES.NB = C.NB AND MILL = 1980)

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin'),
 (44, u'Gide', u'Andre'),
 (45, u'Giono', u'Jean'),
 (8, u'Aragon', u'Louis'),
 (50, u'Lautreamont', None),
 (61, u'Mallarme', u'Stephane')]

#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [16]:
%%sql


SELECT NB, NOM, PRENOM
FROM CUSTOMERS

EXCEPT

SELECT NB, NOM, PRENOM
FROM CUSTOMERS AS C
WHERE 0<(SELECT COUNT(WINES.NV)
        FROM WINES
        JOIN SALES 
        ON WINES.NV = SALES.NV
        WHERE SALES.NB = C.NB AND MILL = 1980)

 * sqlite:///wine.db
Done.


[(1, u'Aristote', None),
 (3, u'Aron', u'Raymond'),
 (4, u'Apollinaire', u'Guillaume'),
 (5, u'Audiberti', u'Jacques'),
 (6, u'Arrabal', u'Fernando'),
 (7, u'Anouilh', u'Jean'),
 (9, u'Ajar', u'Emile'),
 (10, u'Andersen', u'Yann'),
 (11, u'Breton', u'Andre'),
 (12, u'Bataille', u'Georges'),
 (13, u'Barthes', u'Roland'),
 (14, u'Bory', u'Jean Louis'),
 (15, u'Baudelaire', u'Charles'),
 (16, u'Balzac', u'Honore de'),
 (17, u'Camus', u'Albert'),
 (18, u'Celine', u'Louis Ferdinand'),
 (19, u'Ciceron', None),
 (20, u'Chateaubriand', u'Francois-Rene de'),
 (21, u'Corbiere', u'Tristan'),
 (22, u'Cocteau', u'Jean'),
 (23, u'Corneille', u'Pierre'),
 (24, u'Courrier', u'Paul Louis'),
 (25, u'Char', u'Rene'),
 (26, u'Diderot', u'Denis'),
 (27, u'Dumas', u'Alexandre'),
 (28, u'Dostoievsky', u'Fiodor'),
 (29, u'Fournier', u'Alain'),
 (30, u'La Fontaine', u'Jean de'),
 (31, u'Dante', None),
 (32, u'Eluard', u'Paul'),
 (33, u'Flaubert', u'Gustave'),
 (34, u'Mauriac', u'Francois'),
 (35, u'Fromentin', u'Eugene'),
 (36, u'Maurois', u'Andre'),
 (37, u'Freud', u'Sigmund'),
 (38, u'Montaigne', u'Michel de'),
 (39, u'Montesquieu', None),
 (40, u'Moliere', None),
 (41, u'Gauthier', u'Theophile'),
 (42, u'Goethe', u'Johann Wolfgang von'),
 (43, u'Musset', u'Alfred de'),
 (46, u'Hugo', u'Victor'),
 (47, u'Gary', u'Romain'),
 (48, u'Hardy', u'Thomas'),
 (49, u'Kafka', u'Franz'),
 (51, u'Lafargue', u'Paul'),
 (52, u'Carroll', u'Lewis'),
 (53, u'Lamartine', u'Alphonse de'),
 (54, u'Aurele', u'Marc'),
 (55, u'Michelet', u'Jules'),
 (56, u'Nerval', u'Gerard de'),
 (57, u'Marx', u'Karl'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de'),
 (59, u'Nietzsche', u'Friedrich'),
 (60, u'Ovide', None),
 (62, u'Proust', u'Marcel'),
 (63, u'Maupassant', u'Guy de'),
 (64, u'Platon', None),
 (65, u'Pascal', u'Blaise'),
 (66, u'Perec', u'Georges'),
 (67, u'Nin', u'Anais'),
 (68, u'Prevert', u'Jacques'),
 (69, u'Queneau', u'Raymond'),
 (70, u'Paulhan', u'Jean'),
 (71, u'Radiguet', u'Raymond'),
 (72, u'Rabelais', u'Francois'),
 (73, u'Poe', u'Allan Edgard'),
 (74, u'Rimbaud', u'Arthur'),
 (75, u'Ronsard', u'Pierre'),
 (76, u'Racine', u'Jean'),
 (77, u'Rousseau', u'Jean-Jacques'),
 (78, u'Sandier', u'Jules'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de'),
 (80, u'Sand', u'Georges'),
 (81, u'Sartre', u'Jean-Paul'),
 (82, u'Roussel', u'Rene'),
 (83, u'Sophocle', None),
 (84, u'Sue', u'Eugene'),
 (85, u'Segalen', u'Victor'),
 (86, u'Saint Exupery', u'Antoine de'),
 (87, u'Segur', u'Comtesse de'),
 (88, u'Shakespeare', u'William'),
 (89, u'Stendhal', None),
 (90, u'Vian', u'Boris'),
 (91, u'Verlaine', u'Paul'),
 (92, u'Verne', u'Jules'),
 (93, u'Virgile', None),
 (94, u'Zola', u'Emile'),
 (95, u'Zorn', u'Fritz'),
 (96, u'Wilde', u'Oscar'),
 (97, u'Vigny', u'Alfred de'),
 (98, u'Maturin', u'Robert'),
 (99, u'Cesar', u'Jules'),
 (100, u'Woolf', u'Virginia')]

#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [17]:
%%sql

SELECT NB, NOM, PRENOM
FROM CUSTOMERS AS C
WHERE 0<(SELECT COUNT(WINES.NV)
        FROM WINES
        JOIN SALES 
        ON WINES.NV = SALES.NV
        WHERE SALES.NB = C.NB AND MILL = 1980)

EXCEPT

SELECT NB, NOM, PRENOM
FROM CUSTOMERS AS C
WHERE 0<(SELECT COUNT(WINES.NV)
        FROM WINES
        JOIN SALES 
        ON WINES.NV = SALES.NV
        WHERE SALES.NB = C.NB AND MILL != 1980)


 * sqlite:///wine.db
Done.


[(44, u'Gide', u'Andre'), (45, u'Giono', u'Jean'), (50, u'Lautreamont', None)]

#### Exercise 2.10

List all wines from 1980

In [18]:
%%sql

SELECT *
FROM WINES
WHERE MILL = 1980;

 * sqlite:///wine.db
Done.


[(1, u'Mercurey', 11.5, 1980),
 (4, u'Mercurey', 10.9, 1980),
 (16, u'Meursault', 12.1, 1980),
 (20, u'Cote de Brouilly', 12.1, 1980),
 (26, u'Chateau Corton Grancey', None, 1980),
 (28, u'Volnay', 11, 1980),
 (43, u'Fleurie', 11.4, 1980),
 (74, u'Arbois', 12, 1980),
 (78, u'Etoile', 12, 1980),
 (79, u'Seyssel', 11, 1980),
 (80, u'Seyssel', 11, 1980),
 (81, u'Cornas', 11.1, 1980),
 (82, u'Cornas', 11, 1980),
 (83, u'Saint Amour', 10, 1980),
 (84, u'Rasteau', 10, 1980),
 (85, u'Tavel', 11, 1980),
 (86, u'Tavel', 11, 1980),
 (87, u'Cotes de Provence', None, 1980)]

#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [19]:
%%sql



SELECT WINES.NV, CRU, MILL, DEGRE
FROM WINES
JOIN SALES
ON WINES.NV = SALES.NV
WHERE NB = 2 AND MILL = 1980;


--Another way to do it, more effective.
SELECT WNV, CRU, MILL, DEGRE
FROM (SELECT NV AS WNV, CRU, MILL, DEGRE
    FROM WINES
    WHERE MILL = 1980),
    (SELECT NV AS SNV, NB
    FROM SALES
    WHERE NB = 2)
WHERE SNV = WNV;

 * sqlite:///wine.db
Done.
Done.


[(1, u'Mercurey', 1980, 11.5)]

#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [20]:
%%sql

SELECT NB, NOM, PRENOM
FROM CUSTOMERS AS C
WHERE (SELECT COUNT(NV)
       FROM WINES
       WHERE MILL = 1980) = (SELECT COUNT(WNV)
                             FROM (SELECT NV AS WNV
                                   FROM WINES
                                   WHERE MILL = 1980),
                                  (SELECT NV AS SNV, NB
                                   FROM SALES
                                   WHERE NB = C.NB)
                             WHERE SNV = WNV);

 * sqlite:///wine.db
Done.


[(44, u'Gide', u'Andre')]